**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

## Author: MONTALVO Ana - DSBA 2018

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [17]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?


The function act returns the next action that the agent will do depending on the given state. The action is based on the previous knowledge that the algorithm has already accumulated during the random training, where epsilon is the parameter that controls the exploration.
The current algorithm is using the e-greedy strategy, where the epsilon value is essential, because the next action that the algorithm will take is based on it. In this case, if the random number generated is greater than epsilon, then the next action will look for an action based on the learned_act function. But, if the random number is lower or equal than epsilon, then the next act that the function will generate is random.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [18]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1   #Corrected

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [19]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=25 # set small when debugging
epochs_test=25 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

___position___: In this array, each cell (x,y) represents the cells where the rat can go. This array contains the value -1 where the rat can't go, the value 0 where the rat can go, and the value 1 where the rat is at the current state.

___board___:    In this array, each cell (x,y) represents the reward that the rat can get if it goes to this position of the board.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [20]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [21]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state    
        state = env.reset()
        # This assumes that the games will end
        game_over = False
        # We start the variables win and lose to acumulate the rewards gotten during the training
        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win  = win + reward
            if reward < 0:
                lose = lose - reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Epoch {}: Win/lose count {}/{}. Average score ({})"
              .format(e+1, win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [22]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Epoch 1: Win/lose count 11.5/14.0. Average score (-2.5)
Epoch 2: Win/lose count 12.5/15.0. Average score (-2.5)
Epoch 3: Win/lose count 10.0/15.0. Average score (-3.3333333333333335)
Epoch 4: Win/lose count 9.5/10.0. Average score (-2.625)
Epoch 5: Win/lose count 5.5/10.0. Average score (-3.0)
Epoch 6: Win/lose count 10.5/13.0. Average score (-2.9166666666666665)
Epoch 7: Win/lose count 8.0/19.0. Average score (-4.071428571428571)
Epoch 8: Win/lose count 10.0/11.0. Average score (-3.6875)
Epoch 9: Win/lose count 8.5/12.0. Average score (-3.6666666666666665)
Epoch 10: Win/lose count 8.0/18.0. Average score (-4.3)
Epoch 11: Win/lose count 7.5/9.0. Average score (-4.045454545454546)
Epoch 12: Win/lose count 8.5/17.0. Average score (-4.416666666666667)
Epoch 13: Win/lose count 11.5/12.0. Average score (-4.115384615384615)
Epoch 14: Win/lose count 8.0/18.0. Average score (-4.535714285714286)
Epoch 15: Win/lose count 7.5/20.0. Average score (-5.066666666666666)
Epoch 16: Win/lose count 12.5/

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




## Answer: 

We have:

\begin{align}
Q^{\pi}(s,a)&=E[ \sum_{k=0}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& = E[ r(s,a) + \sum_{k=1}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& =  r(s,a) + \gamma E[E[\sum_{k=0}^{\infty}[\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']|s_t = s, a_t = a] \\
& =  r(s,a) + \gamma \sum_{(s',a')}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )[E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
& =  \sum_{(s',a')} (p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) r(s,a) + \gamma \sum_{(s',a')}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}

Then we have:

\begin{align}
Q^{*}(s,a) &= \max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
&= \max_{\pi}r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi}(s',a')\\
&= r(s,a) + \gamma \max_{\pi} \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \qquad \text{with } \pi=(a',\pi')
\end{align}

In order to exchange the maximum and the sum one has to remark the following:
We have first the trivial inequality

\begin{equation*}
\max_{\pi} \sum_{(s')} p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \leq \sum_{(s',a')} p(s_{t+1}=s' |s_t = s, a_t = a ) \max_{\pi}Q^{\pi'}(s',a') 
\end{equation*}

Then we have considering $\hat{\pi}(s',a') = \arg\max_{\pi'} Q^{\pi'}(s',a')$
\begin{align*}
 \sum_{(s'} p(s_{t+1}=s'|s_t = s, a_t = a ) \max_{\pi'}Q^{\pi'}(s',a') &= \sum_{(s')} p(s_{t+1}=s' |s_t = s, a_t = a ) Q^{\hat{\pi}(s',a')}(s',a') \\
 &\leq \max_{\pi'} \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') 
\end{align*}

So we can rewrite our initial equation as :
\begin{align}
Q^{*}(s,a)&= r(s,a) + \gamma  \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'}\max_{\pi'}Q^{\pi'}(s',a') \\
&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'} Q^{*}(s',a') \\ 
&=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{align}

The proposed objective function 

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

Allows to enforce the previous equality, if our neural network perfectly converge this equality must be true. Using this loss function we try to enforce our learned Q-value to verify the Bellman equation.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [70]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)>=self.max_memory:
            del self.memory[0]
        self.memory.append(m)

    def random_access(self):
        index = np.random.randint(0,len(self.memory),size=1)[0]
        return self.memory[index]

***
The pipeline we will use for training is given below:

In [71]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
    

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
    

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [72]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99 
        self.grid_size = grid_size
        # number of state
        self.n_state = n_state
        # Memory
        self.memory = Memory(memory_size)
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # Returns the best optimizer using the previous actions already saved
        optimizer = np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])
        return optimizer

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])     
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            # We call a random act saved in the memory class
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            
            # In this step, we will try to improve the previous prediction,
            # so if we don't have new information, we keep the previous prediction.
            if game_over_:
                input_states[i] = s_
                target_q[i,a_] = r_
            else:
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        # A one layer method is tried here
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(50,activation ='relu'))
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [74]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.5, epsilon = 0.1, memory_size=2000, batch_size = 20)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 001/025 | Loss 0.0020 | Win/lose count 3.5/6.0 (-2.5)
Epoch 002/025 | Loss 0.0020 | Win/lose count 3.5/1.0 (2.5)
Epoch 003/025 | Loss 0.0023 | Win/lose count 4.0/1.0 (3.0)
Epoch 004/025 | Loss 0.0028 | Win/lose count 4.0/5.0 (-1.0)
Epoch 005/025 | Loss 0.0012 | Win/lose count 3.5/3.0 (0.5)
Epoch 006/025 | Loss 0.0040 | Win/lose count 6.0/4.0 (2.0)
Epoch 007/025 | Loss 0.0021 | Win/lose count 5.5/5.0 (0.5)
Epoch 008/025 | Loss 0.0142 | Win/lose count 5.5/3.0 (2.5)
Epoch 009/025 | Loss 0.0024 | Win/lose count 8.5/0 (8.5)
Epoch 010/025 | Loss 0.0905 | Win/lose count 0.5/1.0 (-0.5)
Epoch 011/025 | Loss 0.0015 | Win/lose count 5.5/2.0 (3.5)
Epoch 012/025 | Loss 0.0015 | Win/lose count 8.0/1.0 (7.0)
Epoch 013/025 | Loss 0.0598 | Win/lose count 8.0/2.0 (6.0)
Epoch 014/025 | Loss 0.0019 | Win/lose count 3.0/2.0 (1.0)
Epoch 015/025 | Loss 0.0024 | Win/lose count 5.5/4.0 (1.5)
Epoch 016/025 | Loss 0.0008 | Win/lose count 8.0/1.0 (7.0)
Epoch 017/025 | Loss 0.0027 | Win/lose count 11.5/2.0 (

## Comparison of several neural networks changing parameters:

For this part, I tried to change the parameters: Learning Rate(LR), Epsilon, Memory size, and the batch size.

Architecture 1: One Layer Method

We find that the best score is got when we use the following parameters:
**lr=.5, epsilon = 0.1, memory_size=2000, batch_size = 20**

**EXPERIMENT'S RESULTS:**

|Architecture | # Epochs | LR | Epsilon | Memory size| Batch size | Avg Score | Std Desv. 
| :------ | :------: | ------: | ------: |------: |------: |------: |------: |
| 1 | 25 | 0.1 | 0.1 | 2000 | 32 |  0.42 | 2.75
| 1 | 25 | 0.1 | 0.1 | 5000 | 32 |  0.82 | 2.54
| 1 | 25 | 0.1 | 0.5 | 2000 | 32 | -0.20 | 4.39
| 1 | 25 | 0.1 | 0.5 | 5000 | 32 |  0.46 | 4.17
| 1 | 25 | 0.5 | 0.1 | 2000 | 32 |  1.74 | 2.56
| 1 | 25 | 0.5 | 0.1 | 5000 | 32 |  1.24 | 2.27
| 1 | 25 | 0.5 | 0.5 | 2000 | 32 |  1.66 | 3.90
| 1 | 25 | 0.5 | 0.5 | 5000 | 32 |  2.02 | 4.54
| 1 | 25 | 0.1 | 0.1 | 2000 | 20 |  2.82 | 3.36
| 1 | 25 | 0.1 | 0.1 | 5000 | 20 |  0.82 | 2.21
| 1 | 25 | 0.1 | 0.5 | 2000 | 20 |  2.58 | 3.57
| 1 | 25 | 0.1 | 0.5 | 5000 | 20 |  2.26 | 3.43
| 1 | 25 | 0.5 | 0.1 | 2000 | 20 |  3.12 | 3.17
| 1 | 25 | 0.5 | 0.1 | 5000 | 20 |  2.16 | 4.38
| 1 | 25 | 0.5 | 0.5 | 2000 | 20 |  1.76 | 3.67
| 1 | 25 | 0.5 | 0.5 | 5000 | 20 |  2.36 | 4.10

__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [76]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(30,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [77]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.5, epsilon = 0.1, memory_size=5000, batch_size = 20)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 001/025 | Loss 0.0013 | Win/lose count 3.5/5.0 (-1.5)
Epoch 002/025 | Loss 0.0075 | Win/lose count 7.5/5.0 (2.5)
Epoch 003/025 | Loss 0.0029 | Win/lose count 1.5/3.0 (-1.5)
Epoch 004/025 | Loss 0.0014 | Win/lose count 4.5/4.0 (0.5)
Epoch 005/025 | Loss 0.0063 | Win/lose count 4.5/3.0 (1.5)
Epoch 006/025 | Loss 0.0013 | Win/lose count 2.0/1.0 (1.0)
Epoch 007/025 | Loss 0.0011 | Win/lose count 11.0/2.0 (9.0)
Epoch 008/025 | Loss 0.0017 | Win/lose count 13.5/2.0 (11.5)
Epoch 009/025 | Loss 0.0010 | Win/lose count 10.5/3.0 (7.5)
Epoch 010/025 | Loss 0.0017 | Win/lose count 6.5/2.0 (4.5)
Epoch 011/025 | Loss 0.0099 | Win/lose count 8.5/2.0 (6.5)
Epoch 012/025 | Loss 0.0025 | Win/lose count 8.0/3.0 (5.0)
Epoch 013/025 | Loss 0.0020 | Win/lose count 5.5/1.0 (4.5)
Epoch 014/025 | Loss 0.0013 | Win/lose count 9.0/3.0 (6.0)
Epoch 015/025 | Loss 0.0015 | Win/lose count 5.0/3.0 (2.0)
Epoch 016/025 | Loss 0.0825 | Win/lose count 7.5/2.0 (5.5)
Epoch 017/025 | Loss 0.0031 | Win/lose count 3.5/2

## Comparison of several neural networks changing parameters:

For this part, I tried to change the parameters: Learning Rate(LR), Epsilon, Memory size, and the batch size 

Architecture 1: Hidden Convolutional Layer (50,2,2), the (30,2,2) with relu activations.

We find that the best score is got when we use the following parameters:
**lr=.5, epsilon = 0.1, memory_size=5000, batch_size = 20**

**EXPERIMENT'S RESULTS**

|Architecture | # Epochs | LR | Epsilon | Memory size| Batch size | Avg Score | Std Desv. 
| :------ | :------: | ------: | ------: |------: |------: |------: |------: |
| 1 | 25 | 0.1 | 0.1 | 2000 | 32 |  4.74 | 5.59
| 1 | 25 | 0.1 | 0.1 | 5000 | 32 |  3.68 | 3.03
| 1 | 25 | 0.1 | 0.5 | 2000 | 32 |  2.58 | 4.51
| 1 | 25 | 0.1 | 0.5 | 5000 | 32 |  2.04 | 3.82
| 1 | 25 | 0.5 | 0.1 | 2000 | 32 |  6.94 | 4.34
| 1 | 25 | 0.5 | 0.1 | 5000 | 32 |  7.18 | 5.06
| 1 | 25 | 0.5 | 0.5 | 2000 | 32 |  1.46 | 4.31
| 1 | 25 | 0.5 | 0.5 | 5000 | 32 |  3.70 | 4.85
| 1 | 25 | 0.1 | 0.1 | 2000 | 20 |  2.90 | 4.09
| 1 | 25 | 0.1 | 0.1 | 5000 | 20 |  5.42 | 5.55
| 1 | 25 | 0.1 | 0.5 | 2000 | 20 |  1.20 | 3.95
| 1 | 25 | 0.1 | 0.5 | 5000 | 20 |  2.32 | 4.03
| 1 | 25 | 0.5 | 0.1 | 2000 | 20 |  5.52 | 4.30
| 1 | 25 | 0.5 | 0.1 | 5000 | 20 |  7.54 | 5.22
| 1 | 25 | 0.5 | 0.5 | 2000 | 20 |  2.70 | 4.32
| 1 | 25 | 0.5 | 0.5 | 5000 | 20 |  1.46 | 3.92



__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [87]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Epoch 1: Win/lose count 9.0/0. Average score (9.0)
Epoch 2: Win/lose count 13.5/0. Average score (11.25)
Epoch 3: Win/lose count 18.0/0. Average score (13.5)
Epoch 4: Win/lose count 1.5/0. Average score (10.5)
Epoch 5: Win/lose count 2.5/0. Average score (8.9)
Epoch 6: Win/lose count 13.0/0. Average score (9.583333333333334)
Epoch 7: Win/lose count 5.5/0. Average score (9.0)
Epoch 8: Win/lose count 11.0/0. Average score (9.25)
Epoch 9: Win/lose count 6.0/0. Average score (8.88888888888889)
Epoch 10: Win/lose count 2.5/0. Average score (8.25)
Epoch 11: Win/lose count 1.5/0. Average score (7.636363636363637)
Epoch 12: Win/lose count 2.0/0. Average score (7.166666666666667)
Epoch 13: Win/lose count 6.5/0. Average score (7.115384615384615)
Epoch 14: Win/lose count 7.5/0. Average score (7.142857142857143)
Epoch 15: Win/lose count 2.0/0. Average score (6.8)
Epoch 16: Win/lose count 7.0/0. Average score (6.8125)
Epoch 17: Win/lose count 5.0/0. Average score (6.705882352941177)

In [88]:
HTML(display_videos('cnn_test10.mp4'))

In [89]:
HTML(display_videos('fc_test10.mp4'))

## Comparison of several neural networks changing temperature:

For this part, I tried to change the parameters: temperature 3 times with the following results:

**EXPERIMENT'S RESULTS**

|Architecture | Temperature | Final Score
| :------ | :------: | ------: | 
| CNN | 0.3 | 1.78  
| CNN | 0.5 | 3.50
| CNN | 0.8 | 7.92 
| FC  | 0.3 | -0.46  
| FC  | 0.5 | 0.16
| FC  | 0.8 | 0.76 

We notice that in all the combinations the CNN model performs better than FC. Also we notice that if we increase the temperature, the final score result increases, but checking the performance in the videos, they show that the rat stays in the same 2 steps for the whole video, and it is not exploring the rest of the map. This happens because from the current position, the rat can't see any other positive reward.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [97]:
def train_explore(agent,env,epoch,prefix=''):
    #New training procedure that tries to improve the exploration of the algorithm
    
    # Number of won games
    score = 0
    loss  = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        # Decreasing epsilon greedy exploration
        agent.set_epsilon(agent.epsilon)
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) #define malus when going to a previously visited position
        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        # In the previous questions, we noticed that the rat is not exploring the whole
        # map. With this function, we are going to improve this part.

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1     
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [98]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Exception ignored in: <bound method ScopedTFStatus.__del__ of <tensorflow.python.framework.c_api_util.ScopedTFStatus object at 0x0000007817835B70>>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 39, in __del__
    c_api.TF_DeleteStatus(self.status)
AttributeError: 'ScopedTFStatus' object has no attribute 'status'


Epoch 001/025 | Loss 0.0114 | Win/lose count 6.0/22.50000000000006 (-16.50000000000006)
Epoch 002/025 | Loss 0.0123 | Win/lose count 3.5/20.50000000000002 (-17.00000000000002)
Epoch 003/025 | Loss 0.0065 | Win/lose count 3.5/18.19999999999999 (-14.699999999999989)
Epoch 004/025 | Loss 0.0076 | Win/lose count 6.0/18.29999999999999 (-12.29999999999999)
Epoch 005/025 | Loss 0.0047 | Win/lose count 13.5/18.5 (-5.0)
Epoch 006/025 | Loss 0.0085 | Win/lose count 17.0/15.89999999999997 (1.1000000000000298)
Epoch 007/025 | Loss 0.0037 | Win/lose count 13.5/19.50000000000002 (-6.000000000000021)
Epoch 008/025 | Loss 0.0119 | Win/lose count 7.5/21.40000000000005 (-13.900000000000048)
Epoch 009/025 | Loss 0.0048 | Win/lose count 14.5/15.299999999999962 (-0.7999999999999616)
Epoch 010/025 | Loss 0.0096 | Win/lose count 8.0/17.299999999999976 (-9.299999999999976)
Epoch 011/025 | Loss 0.0079 | Win/lose count 14.5/16.999999999999982 (-2.4999999999999822)
Epoch 012/025 | Loss 0.0035 | Win/lose count 4.

In [99]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Epoch 1: Win/lose count 15.5/0. Average score (15.5)
Epoch 2: Win/lose count 14.5/3.0. Average score (13.5)
Epoch 3: Win/lose count 16.0/1.0. Average score (14.0)
Epoch 4: Win/lose count 20.5/1.0. Average score (15.375)
Epoch 5: Win/lose count 21.0/0. Average score (16.5)
Epoch 6: Win/lose count 21.0/1.0. Average score (17.083333333333332)
Epoch 7: Win/lose count 8.0/1.0. Average score (15.642857142857142)
Epoch 8: Win/lose count 19.0/0. Average score (16.0625)
Epoch 9: Win/lose count 19.0/2.0. Average score (16.166666666666668)
Epoch 10: Win/lose count 3.5/0. Average score (14.9)
Epoch 11: Win/lose count 17.5/1.0. Average score (15.045454545454545)
Epoch 12: Win/lose count 19.5/2.0. Average score (15.25)
Epoch 13: Win/lose count 23.5/2.0. Average score (15.73076923076923)
Epoch 14: Win/lose count 19.0/3.0. Average score (15.75)
Epoch 15: Win/lose count 18.5/1.0. Average score (15.866666666666667)
Epoch 16: Win/lose count 16.0/1.0. Average score (15.8125)
Epoch 17: Win/lose count 19.0/

**Conclusion:**
The outcome of this approach has improved a lot vs. the previous experiments. Reaching a score of almost 16 with a temperature of 0.3, when in the previous experiments, we had just reached a reward of less than 10.


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [109]:
class DQN_Mimick(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN_Mimick, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_[:,:,1:], a_]) 
        # we just need the current state and the action that has been applied
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, a_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i,a_] = 10  
            #Now we want to mimick the expert, so we want to maximize the
            #target function in the action applied by the expert           
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_CNN_Mimick(DQN_Mimick):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN_Mimick, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu')) #model with 1 hidden CNN performs the best
        #with my method from what I ve experimented
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [110]:
def train_mimick(expert,agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0    
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = expert.act(state,train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [111]:
expert = agent
agent_n = DQN_CNN_Mimick(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimick(expert,agent_n, env, epochs_train, prefix='cnn_train_mimick')

Epoch 001/025 | Loss 4.2876 | Win/lose count 17.0/0 (17.0)
Epoch 002/025 | Loss 3.8049 | Win/lose count 15.5/1.0 (14.5)
Epoch 003/025 | Loss 3.2228 | Win/lose count 12.0/1.0 (11.0)
Epoch 004/025 | Loss 2.4342 | Win/lose count 14.0/0 (14.0)
Epoch 005/025 | Loss 3.6731 | Win/lose count 17.0/0 (17.0)
Epoch 006/025 | Loss 1.5576 | Win/lose count 5.0/0 (5.0)
Epoch 007/025 | Loss 2.7464 | Win/lose count 18.5/1.0 (17.5)
Epoch 008/025 | Loss 1.6740 | Win/lose count 10.0/1.0 (9.0)
Epoch 009/025 | Loss 4.6215 | Win/lose count 19.5/0 (19.5)
Epoch 010/025 | Loss 2.1016 | Win/lose count 3.5/0 (3.5)
Epoch 011/025 | Loss 4.5505 | Win/lose count 8.0/0 (8.0)
Epoch 012/025 | Loss 2.5588 | Win/lose count 24.5/1.0 (23.5)
Epoch 013/025 | Loss 3.4981 | Win/lose count 14.5/2.0 (12.5)
Epoch 014/025 | Loss 3.5765 | Win/lose count 17.0/0 (17.0)
Epoch 015/025 | Loss 2.5674 | Win/lose count 18.0/1.0 (17.0)
Epoch 016/025 | Loss 3.0465 | Win/lose count 5.0/0 (5.0)
Epoch 017/025 | Loss 2.2491 | Win/lose count 5.5/0 

In [112]:
env_test = Environment(grid_size=size, max_time=T,temperature=0.3)
test(agent_n,env_test,epochs_test,prefix='cnn_test_mimick')

Epoch 1: Win/lose count 13.5/3.0. Average score (10.5)
Epoch 2: Win/lose count 0.5/0. Average score (5.5)
Epoch 3: Win/lose count 7.5/0. Average score (6.166666666666667)
Epoch 4: Win/lose count 0.5/0. Average score (4.75)
Epoch 5: Win/lose count 10.5/1.0. Average score (5.7)
Epoch 6: Win/lose count 8.5/2.0. Average score (5.833333333333333)
Epoch 7: Win/lose count 6.0/0. Average score (5.857142857142857)
Epoch 8: Win/lose count 0/1.0. Average score (5.0)
Epoch 9: Win/lose count 11.5/5.0. Average score (5.166666666666667)
Epoch 10: Win/lose count 0.5/0. Average score (4.7)
Epoch 11: Win/lose count 7.0/0. Average score (4.909090909090909)
Epoch 12: Win/lose count 4.5/0. Average score (4.875)
Epoch 13: Win/lose count 2.0/0. Average score (4.653846153846154)
Epoch 14: Win/lose count 5.0/0. Average score (4.678571428571429)
Epoch 15: Win/lose count 8.5/1.0. Average score (4.866666666666666)
Epoch 16: Win/lose count 4.5/2.0. Average score (4.71875)
Epoch 17: Win/lose count 1.5/1.0. Average 

In [108]:
HTML(display_videos('cnn_test_mimick10.mp4'))

**Conclusion:**
The outcome of mimick experiment doesn't improve the result that was found in the previous experiment done in question 10.

***